In [1]:
import pandas as pd

In [2]:
link1 = "./BD/df_basic_name_clean.csv"
link2 = "./BD/df_t_akas_clean.csv"
link3 = "./BD/df_t_basics_clean.csv"
link4 = "./BD/df_t_crew_clean.csv"
link5 = "./BD/df_t_episode_clean.csv"
link6 = "./BD/df_t_principals_clean.csv"
link7 = "./BD/df_t_ratings_clean.csv"
df_basic_name = pd.read_csv(link1, nrows = 10000)
df_t_akas = pd.read_csv(link2, nrows = 10000)
df_t_basics = pd.read_csv(link3, nrows = 10000)
df_t_crew = pd.read_csv(link4, nrows = 10000)
df_t_episode = pd.read_csv(link5, nrows = 10000)
df_t_principals = pd.read_csv(link6, nrows = 10000)
df_t_ratings = pd.read_csv(link7, nrows = 10000)

On veut 2 df: un df avec index tconst et un avec index nconst
1.  Commençons par df_tID

In [ ]:
df_t_basics = 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"['Documentary', 'Short']"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"['Animation', 'Short']"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,NaN,5.0,"['Animation', 'Comedy', 'Romance']"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12.0,"['Animation', 'Short']"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"['Comedy', 'Short']"
...,...,...,...,...,...,...,...,...,...
9995,tt0010135,movie,The Foolish Fish of Sawback,The Foolish Fish of Sawback,0,1919.0,NaN,NaN,['Documentary']
9996,tt0010136,movie,Fools and Their Money,Fools and Their Money,0,1919.0,NaN,50.0,['Comedy']
9997,tt0010137,movie,"For Better, for Worse","For Better, for Worse",0,1919.0,NaN,70.0,['Drama']
9998,tt0010138,short,For Bitter or for Verse,For Bitter or for Verse,0,1919.0,NaN,NaN,"['Animation', 'Comedy', 'Short']"


In [ ]:
df_ptconst = df_t_principals.groupby('tconst').agg(list).reset_index()
df_ptconst = df_ptconst.drop('category', axis = 1)
df_ptconst = df_ptconst.rename(columns = {'tconst' : 'tID', 'nconst' : 'nID_implique'})

In [10]:
df_ptconst

,tID,nID_implique
0,tt0000001,"[nm1588970, nm0005690, nm0005690, nm0374658]"
1,tt0000002,"[nm0721526, nm1335271]"
2,tt0000003,"[nm0721526, nm1770680, nm0721526, nm1335271, n..."
3,tt0000004,"[nm0721526, nm1335271]"
4,tt0000005,"[nm0443482, nm0653042, nm0249379]"
...,...,...
1727,tt0001767,"[nm0681933, nm0601698, nm0035187, nm0408436, n..."
1728,tt0001768,[nm0294276]
1729,tt0001769,"[nm0376396, nm0842239, nm0041858, nm0115524, n..."
1730,tt0001770,"[nm0149600, nm0626370, nm0534221, nm0917726, n..."


In [17]:
df_t_akas = df_t_akas.rename(columns = {'title' : 'Versions_des_titres'})
df_tID = pd.merge(df_ptconst, df_t_akas, how = 'left', left_on='tID', right_on='Versions_des_titres')

In [18]:
df_tID

,tID,nID_implique,titleId,Versions_des_titres
0,tt0000001,"[nm1588970, nm0005690, nm0005690, nm0374658]",NaN,NaN
1,tt0000002,"[nm0721526, nm1335271]",NaN,NaN
2,tt0000003,"[nm0721526, nm1770680, nm0721526, nm1335271, n...",NaN,NaN
3,tt0000004,"[nm0721526, nm1335271]",NaN,NaN
4,tt0000005,"[nm0443482, nm0653042, nm0249379]",NaN,NaN
...,...,...,...,...
1727,tt0001767,"[nm0681933, nm0601698, nm0035187, nm0408436, n...",NaN,NaN
1728,tt0001768,[nm0294276],NaN,NaN
1729,tt0001769,"[nm0376396, nm0842239, nm0041858, nm0115524, n...",NaN,NaN
1730,tt0001770,"[nm0149600, nm0626370, nm0534221, nm0917726, n...",NaN,NaN
